# Recognize named entities on Twitter with LSTMs

In this assignment, you will use a recurrent neural network to solve Named Entity Recognition (NER) problem. NER is a common task in natural language processing systems. It serves for extraction such entities from the text as persons, organizations, locations, etc. In this task you will experiment to recognize named entities from Twitter.

For example, we want to extract persons' and organizations' names from the text. Than for the input text:

    Ian Goodfellow works for Google Brain

a NER model needs to provide the following sequence of tags:

    B-PER I-PER    O     O   B-ORG  I-ORG

Where *B-* and *I-* prefixes stand for the beginning and inside of the entity, while *O* stands for out of tag or no tag. Markup with the prefix scheme is called *BIO markup*. This markup is introduced for distinguishing of consequent entities with similar types.

A solution of the task will be based on neural networks, particularly, on Bi-Directional Long Short-Term Memory Networks (Bi-LSTMs).

### Libraries

For this task you will need the following libraries:
 - [Tensorflow](https://www.tensorflow.org) — an open-source software library for Machine Intelligence.
 - [Numpy](http://www.numpy.org) — a package for scientific computing.
 
If you have never worked with Tensorflow, you would probably need to read some tutorials during your work on this assignment, e.g. [this one](https://www.tensorflow.org/tutorials/recurrent) could be a good starting point. 

### Data

The following cell will download all data required for this assignment into the folder `week2/data`.

In [1]:
! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
import setup_google_colab
# please, uncomment the week you're working on
# setup_google_colab.setup_week1()  
setup_google_colab.setup_week2()
# setup_google_colab.setup_week3()
# setup_google_colab.setup_week4()
# setup_google_colab.setup_project()
# setup_google_colab.setup_honor()

--2018-12-21 01:09:46--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2330 (2.3K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   2.28K  --.-KB/s    in 0s      

2018-12-21 01:09:46 (27.7 MB/s) - ‘setup_google_colab.py’ saved [2330/2330]



In [2]:
import sys
sys.path.append("..")
from common.download_utils import download_week2_resources

download_week2_resources()

### Load the Twitter Named Entity Recognition corpus

We will work with a corpus, which contains tweets with NE tags. Every line of a file contains a pair of a token (word/punctuation symbol) and a tag, separated by a whitespace. Different tweets are separated by an empty line.

The function *read_data* reads a corpus from the *file_path* and returns two lists: one with tokens and one with the corresponding tags. You need to complete this function by adding a code, which will replace a user's nickname to `<USR>` token and any URL to `<URL>` token. You could think that a URL and a nickname are just strings which start with *http://* or *https://* in case of URLs and a *@* symbol for nicknames.

In [0]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            # Replace all urls with <URL> token
            # Replace all users with <USR> token

            if(token.startswith('@')):
                token = '<USR>'
                
            elif(token.startswith('http://') or token.startswith('https://')):
                token = '<URL>'
                
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

And now we can load three separate parts of the dataset:
 - *train* data for training the model;
 - *validation* data for evaluation and hyperparameters tuning;
 - *test* data for final evaluation of the model.

In [0]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

You should always understand what kind of data you deal with. For this purpose, you can print the data running the following cell:

In [5]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



### Prepare dictionaries

To train a neural network, we will use two mappings: 
- {token}$\to${token id}: address the row in embeddings matrix for the current token;
- {tag}$\to${tag id}: one-hot ground truth probability distribution vectors for computing the loss at the output of the network.

Now you need to implement the function *build_dict* which will return {token or tag}$\to${index} and vice versa. 

In [0]:
from collections import defaultdict

In [0]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens to indices and vice versa
    # Add special tokens to dictionaries
    # The first special token must have index 0
    
    idx = 0
    for token in special_tokens:
        tok2idx[token] = idx
        idx2tok.append(token)
        idx += 1

    for token_list in tokens_or_tags:
        for token in token_list:
            if token not in tok2idx.keys():
                tok2idx[token] = idx
                idx2tok.append(token)
                idx += 1
    
    return tok2idx, idx2tok

After implementing the function *build_dict* you can make dictionaries for tokens and tags. Special tokens in our case will be:
 - `<UNK>` token for out of vocabulary tokens;
 - `<PAD>` token for padding sentence to the same length when we create batches of sentences.

In [0]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

The next additional functions will help you to create the mapping between tokens and ids for a sentence. 

In [0]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

## Build a recurrent neural network

This is the most important part of the assignment. Here we will specify the network architecture based on TensorFlow building blocks. It's fun and easy as a lego constructor! We will create an LSTM network which will produce probability distribution over tags for each token in a sentence. To take into account both right and left contexts of the token, we will use Bi-Directional LSTM (Bi-LSTM). Dense layer will be used on top to perform tag classification.  

In [0]:
import tensorflow as tf
import numpy as np
from evaluation import precision_recall_f1

In [0]:
class BiLSTMModel():
    def __init__(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
        self.create_placeholders()
        self.build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags);
        self.compute_predictions()
        self.compute_loss(n_tags, PAD_index)
        self.create_optimizer()
        
    def create_placeholders(self):
        
        # Placeholders for input and ground truth output.
        self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
        self.ground_truth_tags = self.ground_truth_tags = tf.placeholder(dtype=tf.int32, shape=[None, None], name='ground_truth_tags')

        # Placeholder for lengths of the sequences.
        self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name='lengths') 

        # Placeholder for a dropout keep probability. If we don't feed
        # a value for this placeholder, it will be equal to 1.0.
        self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])

        # Placeholder for a learning rate (tf.float32).
        self.learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[], name='learning_rate_ph')
    
    def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
        # Create embedding variable (tf.Variable) with dtype tf.float32
        initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
        embedding_matrix_variable = tf.Variable(initial_embedding_matrix, name='embeddings_matrix', dtype=tf.float32)
        embeddings = tf.nn.embedding_lookup(embedding_matrix_variable, self.input_batch)
  

        forward_cell = tf.nn.rnn_cell.DropoutWrapper(
            tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn, forget_bias=3.0),
            input_keep_prob=self.dropout_ph,
            output_keep_prob=self.dropout_ph,
            state_keep_prob=self.dropout_ph
        )
        backward_cell = tf.nn.rnn_cell.DropoutWrapper(
            tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn, forget_bias=3.0),
            input_keep_prob=self.dropout_ph,
            output_keep_prob=self.dropout_ph,
            state_keep_prob=self.dropout_ph
        )
        
        (rnn_output_fw, rnn_output_bw), _ = tf.nn.bidirectional_dynamic_rnn(
        cell_fw= forward_cell, cell_bw= backward_cell,
        dtype=tf.float32,
        inputs=embeddings,
        sequence_length=self.lengths
        )
        rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)
        self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)
    
    def compute_loss(self, n_tags, PAD_index):
        ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
        loss_tensor = tf.nn.softmax_cross_entropy_with_logits_v2(labels = ground_truth_tags_one_hot,logits = self.logits)

        # Create loss function which doesn't operate with <PAD> tokens (tf.reduce_mean)
        mask = tf.cast(tf.not_equal(loss_tensor, PAD_index), tf.float32)
        loss_tensor = tf.contrib.seq2seq.sequence_loss(logits=self.logits,targets=self.ground_truth_tags,weights=mask)
        self.loss =  tf.reduce_mean(loss_tensor)
        
    def create_optimizer(self):
        # Create an optimizer (tf.train.AdamOptimizer)
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate_ph)
        self.grads_and_vars = self.optimizer.compute_gradients(self.loss)

        # Gradient clipping (tf.clip_by_norm) for self.grads_and_vars
        # Pay attention that you need to apply this operation only for gradients 
        # because self.grads_and_vars contains also variables.
        # list comprehension might be useful in this case.
        clip_norm = tf.cast(1.0, tf.float32)
        self.grads_and_vars = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in self.grads_and_vars]

        self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)
    
    def compute_predictions(self):
        # Create softmax (tf.nn.softmax) function
        softmax_output = tf.nn.softmax(self.logits)

        # Use argmax (tf.argmax) to get the most probable tags
        # Don't forget to set axis=-1
        # otherwise argmax will be calculated in a wrong way
        self.predictions = tf.argmax(softmax_output, axis=-1)
    
    def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
        feed_dict = {self.input_batch: x_batch,
                     self.ground_truth_tags: y_batch,
                     self.learning_rate_ph: learning_rate,
                     self.dropout_ph: dropout_keep_probability,
                     self.lengths: lengths}

        _, loss = session.run((self.train_op, self.loss), feed_dict=feed_dict)
        return loss
    
    def predict_for_batch(self, session, x_batch, lengths):
        predictions = session.run(self.predictions, feed_dict={self.input_batch:x_batch, self.lengths:lengths})
        return predictions

In [0]:
def batch_generator(batch_size, tokens, tags, shuffle=True, allow_small_last_batch=True):
    num_samples = len(tokens)
    if shuffle:
        sample_idxs = np.random.permutation(num_samples)
    else:
        sample_idxs = np.arange(num_samples)
        
    num_batches = num_samples//batch_size
    
    if allow_small_last_batch and num_samples%batch_size:
        num_batches += 1
    
    for k in range(num_batches):
        batch_start = k * batch_size
        batch_end = min((k+1) * batch_size, num_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        
        for idx in sample_idxs[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
        
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            l = len(x_list[n])
            x[n, :l] = x_list[n]
            y[n, :l] = y_list[n]
            lengths[n] = l
        yield x, y, lengths

In [0]:
def predict_tags (model, session, token_idxs_batch, lengths):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batch_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

In [23]:
tf.reset_default_graph()
model = BiLSTMModel(20505, 21, 200, 200, token2idx['<PAD>'])

batch_size = 32
num_epochs = 4
learning_rate = 0.005
learning_rate_decay = 1.414
dropout_keep_prob = 0.5

sess = tf.Session()
sess.run(tf.global_variables_initializer())

step = 0
for epoch in range(num_epochs):
    for x_batch, y_batch, lengths in batch_generator(batch_size, train_tokens, train_tags):
        
        step += 1
        loss = model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_prob)

    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(num_epochs) + '-' * 20)
    print('Training loss: ', loss)
    print('Train data evaluation:')
    eval_conll(model, sess, train_tokens, train_tags, short_report=True)
    
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_tokens, validation_tags, short_report=True)
    
    learning_rate = learning_rate/learning_rate_decay
    
print('...training finished.')

-------------------- Epoch 1 of 4 --------------------
Training loss:  0.647101
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 2494 phrases; correct: 596.

precision:  23.90%; recall:  13.28%; F1:  17.07

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 203 phrases; correct: 47.

precision:  23.15%; recall:  8.75%; F1:  12.70

-------------------- Epoch 2 of 4 --------------------
Training loss:  0.18938679
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 4745 phrases; correct: 1910.

precision:  40.25%; recall:  42.55%; F1:  41.37

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 343 phrases; correct: 138.

precision:  40.23%; recall:  25.70%; F1:  31.36

-------------------- Epoch 3 of 4 --------------------
Training loss:  0.104392774
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 4935 phrases; correct: 2864.

precision:  58.03%; recall:  63.80%; F1:  60.

To compute the actual predictions of the neural network, you need to apply [softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) to the last layer and find the most probable tags with [argmax](https://www.tensorflow.org/api_docs/python/tf/argmax).

Now let us see full quality reports for the final model on train, validation, and test sets. To give you a hint whether you have implemented everything correctly, you might expect F-score about 40% on the validation set.

**The output of the cell below (as well as the output of all the other cells) should be present in the notebook for peer2peer review!**

In [24]:
print('-' * 20 + ' Train set quality: ' + '-' * 20)
train_results = eval_conll(model, sess, train_tokens, train_tags, short_report=False)

print('-' * 20 + ' Validation set quality: ' + '-' * 20)
validation_results = eval_conll(model, sess, validation_tokens, validation_tags, short_report=False)

print('-' * 20 + ' Test set quality: ' + '-' * 20)
test_results = eval_conll(model, sess, test_tokens, test_tags, short_report=False)

-------------------- Train set quality: --------------------
processed 105778 tokens with 4489 phrases; found: 4653 phrases; correct: 3267.

precision:  70.21%; recall:  72.78%; F1:  71.47

	     company: precision:   79.18%; recall:   81.03%; F1:   80.09; predicted:   658

	    facility: precision:   60.62%; recall:   68.15%; F1:   64.17; predicted:   353

	     geo-loc: precision:   77.68%; recall:   91.57%; F1:   84.06; predicted:  1174

	       movie: precision:    0.00%; recall:    0.00%; F1:    0.00; predicted:     1

	 musicartist: precision:   34.88%; recall:    6.47%; F1:   10.91; predicted:    43

	       other: precision:   61.72%; recall:   76.88%; F1:   68.47; predicted:   943

	      person: precision:   72.57%; recall:   92.89%; F1:   81.49; predicted:  1134

	     product: precision:   53.42%; recall:   51.57%; F1:   52.48; predicted:   307

	  sportsteam: precision:   90.00%; recall:   16.59%; F1:   28.02; predicted:    40

	      tvshow: precision:    0.00%; recall:  

### Conclusions

Could we say that our model is state of the art and the results are acceptable for the task? Definately, we can say so. Nowadays, Bi-LSTM is one of the state of the art approaches for solving NER problem and it outperforms other classical methods. Despite the fact that we used small training corpora (in comparison with usual sizes of corpora in Deep Learning), our results are quite good. In addition, in this task there are many possible named entities and for some of them we have only several dozens of trainig examples, which is definately small. However, the implemented model outperforms classical CRFs for this task. Even better results could be obtained by some combinations of several types of methods, e.g. see [this](https://arxiv.org/abs/1603.01354) paper if you are interested.